<a href="https://colab.research.google.com/github/MiguelUrenaPliego/GeoimageVision/blob/main/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Only execute this cell if you are using google colab
!pip uninstall tensorflow -y
!pip uninstall numba -y
!pip uninstall thic -y
!pip uninstall pytensor -y
!pip install geopandas
!pip install folium matplotlib mapclassify
!pip install ipyleaflet
!pip install torch torchmetrics
#!pip install pytorch-lightning
!pip install lightning
#!pip install lightning-bolts
!pip install git+https://github.com/GeomaticsCaminosUPM/GeoVisionDataset.git
!pip install git+https://github.com/Rusteam/segmentation_models.pytorch.git@sam
!pip install kornia[x]
# After this restart the session

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: numba 0.60.0
Uninstalling numba-0.60.0:
  Successfully uninstalled numba-0.60.0
Found existing installation: pytensor 2.27.1
Uninstalling pytensor-2.27.1:
  Successfully uninstalled pytensor-2.27.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [2]:
import lightning

/usr/local/lib/python3.11/dist-packages/numpy/_core/_dtype.py:106: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if dtype.type == np.bool:


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [2]:
!git clone https://github.com/GeomaticsCaminosUPM/GeoVisionModels.git

Cloning into 'GeoVisionModels'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 66 (delta 25), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 3.02 MiB | 4.51 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [3]:
import GeoVisionModels.code.seg_utils as utils
import GeoVisionModels.code.seg_losses as losses
import GeoVisionModels.code.SemanticSegmentation as Seg

import lightning.pytorch as pl
import torch, torchmetrics
import numpy as np
import os
import segmentation_models_pytorch as smp
from GeoVisionModels.code.seg_utils import SegmentationTransforms

/usr/local/lib/python3.11/dist-packages/numpy/_core/_dtype.py:106: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if dtype.type == np.bool:


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [ ]:
encoder="sam-vit_b"
weights = "sa-1b" # input size must be 1024 for weights sa-1b
input_size = (1024, 1024) # input size must be square
batch_size = 2
augment_rate = 4
num_workers = 1
background_index = 0

max_epochs = 10

mean=[0.44368124, 0.4537536, 0.45856014]
std=[0.1987232, 0.17570017, 0.15577587]

transforms = SegmentationTransforms.Compose([
    SegmentationTransforms.Resize(input_size),
    utils.aug_transforms_2,
    SegmentationTransforms.toTensor(),
    SegmentationTransforms.Normalize(mean,std),
    SegmentationTransforms.toTensor()
])

transforms_val = SegmentationTransforms.Compose([
    SegmentationTransforms.Resize(input_size),
    SegmentationTransforms.toTensor(),
    SegmentationTransforms.Normalize(mean,std),
    SegmentationTransforms.toTensor()
])